# Results

In [1]:
from pathlib import Path
import pandas as pd
from src.helpers.metrics import  build_metrics_table

from experiment_config import DegModel,DATA_NAME,FILTERED_SUFFIX,PFNET_NAME,N_REP



In [2]:
degmodel_name = DegModel.name()


experiment_dir = Path('experiments')/DATA_NAME
pfnet_dir = experiment_dir/f'degradation{FILTERED_SUFFIX}'/degmodel_name/'pf_performs'/PFNET_NAME

## Load Predictions

In [3]:
df_preds = pd.read_csv(pfnet_dir / f"rul_test_predictions_rep{N_REP}.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'experiments/DS03/degradation_filtered/gamma/pf_performs/net64x32leaky0.05_emaloss9/rul_test_predictions_rep10.csv'

## Compute mean prediction

In [ ]:
df_preds = df_preds.groupby(["unit", "time"], as_index=False).mean()

## Full-life Predictions

In [ ]:
metrics_df = build_metrics_table(df_preds)
metrics_df

,unit,RMSE,Score,PICP,PINAW
0,10,8.302299,52.156127,1.000000,0.468794
1,11,6.276366,43.224715,1.000000,0.568825
2,12,20.497773,410.821715,0.258065,0.298462
3,13,13.343120,135.927617,0.831169,0.355510
4,14,13.747131,142.997242,0.973684,0.411180
5,15,11.354529,89.969510,1.000000,0.450590


In [ ]:
metrics_df.drop(columns=["unit"]).mean()

RMSE      12.253536
Score    145.849488
PICP       0.843820
PINAW      0.425560
dtype: float64

## Degradation Predictions

In [ ]:
hi_df = pd.read_csv(experiment_dir/f'hidata_test{FILTERED_SUFFIX}.csv')
test_units = hi_df['unit'].astype(int).unique().tolist()
onsets = {unit: hi_df[(hi_df['unit']==unit) & (hi_df['hs']==0)]['cycle'].values[0] for unit in test_units}
df_degpreds = df_preds[
    df_preds["time"] > df_preds["unit"].map(onsets)
]

In [ ]:
metrics_df = build_metrics_table(df_degpreds)
metrics_df

,unit,RMSE,Score,PICP,PINAW
0,10,5.198503,18.882165,1.000000,0.472168
1,11,3.391975,11.259428,1.000000,0.574086
2,12,11.597056,76.960396,0.321429,0.262534
3,13,9.680282,57.536265,0.793103,0.310930
4,14,1.984789,6.227895,1.000000,0.395783
5,15,4.325151,12.501676,1.000000,0.424443


In [ ]:
metrics_df.drop(columns=["unit"]).mean()

RMSE      6.029626
Score    30.561304
PICP      0.852422
PINAW     0.406657
dtype: float64

## Benchmark Predictions

In [ ]:
df_benchmark = df_preds[df_preds["true_rul"] <= 65]

In [ ]:
metrics_df = build_metrics_table(df_benchmark)
metrics_df

,unit,RMSE,Score,PICP,PINAW
0,10,8.302299,52.156127,1.000000,0.468794
1,11,6.276366,43.224715,1.000000,0.568825
2,12,16.386553,186.225901,0.272727,0.264281
3,13,12.223747,98.744779,0.803030,0.327982
4,14,12.295700,99.610706,0.969697,0.402340
5,15,11.431947,89.545489,1.000000,0.452808


In [ ]:
metrics_df.drop(columns=["unit"]).mean()

RMSE     11.152768
Score    94.917953
PICP      0.840909
PINAW     0.414172
dtype: float64

# DS01 (normal-filtered-full life)
<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>unit</th>
      <th>RMSE</th>
      <th>Score</th>
      <th>PICP</th>
      <th>PINAW</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>7</td>
      <td>10.172457</td>
      <td>102.308087</td>
      <td>1.000000</td>
      <td>0.380963</td>
    </tr>
    <tr>
      <th>1</th>
      <td>8</td>
      <td>7.281235</td>
      <td>62.246042</td>
      <td>0.820225</td>
      <td>0.300238</td>
    </tr>
    <tr>
      <th>2</th>
      <td>9</td>
      <td>4.808850</td>
      <td>42.925448</td>
      <td>0.900000</td>
      <td>0.385455</td>
    </tr>
    <tr>
      <th>3</th>
      <td>10</td>
      <td>2.710854</td>
      <td>16.364241</td>
      <td>1.000000</td>
      <td>0.314442</td>
    </tr>
  </tbody>
</table>
</div>

RMSE      6.243349\
Score    55.960955\
PICP      0.930056\
PINAW     0.345274

# DS01 (normal-filtered-degradation)

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>unit</th>
      <th>RMSE</th>
      <th>Score</th>
      <th>PICP</th>
      <th>PINAW</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>7</td>
      <td>7.623021</td>
      <td>39.824469</td>
      <td>1.000000</td>
      <td>0.477663</td>
    </tr>
    <tr>
      <th>1</th>
      <td>8</td>
      <td>7.641881</td>
      <td>46.288805</td>
      <td>0.746032</td>
      <td>0.327433</td>
    </tr>
    <tr>
      <th>2</th>
      <td>9</td>
      <td>5.875044</td>
      <td>33.014507</td>
      <td>0.822222</td>
      <td>0.472829</td>
    </tr>
    <tr>
      <th>3</th>
      <td>10</td>
      <td>2.335743</td>
      <td>10.715747</td>
      <td>1.000000</td>
      <td>0.339292</td>
    </tr>
  </tbody>
</table>
</div>

RMSE      5.868922\
Score    32.460882\
PICP      0.892063\
PINAW     0.404304

# DS01 (normal-filtered-benchmark)

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>unit</th>
      <th>RMSE</th>
      <th>Score</th>
      <th>PICP</th>
      <th>PINAW</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>7</td>
      <td>9.224528</td>
      <td>64.074776</td>
      <td>1.000000</td>
      <td>0.455816</td>
    </tr>
    <tr>
      <th>1</th>
      <td>8</td>
      <td>7.706186</td>
      <td>49.260934</td>
      <td>0.757576</td>
      <td>0.328729</td>
    </tr>
    <tr>
      <th>2</th>
      <td>9</td>
      <td>5.048802</td>
      <td>37.351187</td>
      <td>0.878788</td>
      <td>0.433838</td>
    </tr>
    <tr>
      <th>3</th>
      <td>10</td>
      <td>2.326541</td>
      <td>11.160553</td>
      <td>1.000000</td>
      <td>0.337296</td>
    </tr>
  </tbody>
</table>
</div>

RMSE      6.076514\
Score    40.461863\
PICP      0.909091\
PINAW     0.388920

# DS03 (normal-filtered-full life)
<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>unit</th>
      <th>RMSE</th>
      <th>Score</th>
      <th>PICP</th>
      <th>PINAW</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>10</td>
      <td>4.190528</td>
      <td>26.301490</td>
      <td>1.000000</td>
      <td>0.509070</td>
    </tr>
    <tr>
      <th>1</th>
      <td>11</td>
      <td>10.147794</td>
      <td>95.884897</td>
      <td>1.000000</td>
      <td>0.586171</td>
    </tr>
    <tr>
      <th>2</th>
      <td>12</td>
      <td>13.850840</td>
      <td>170.770287</td>
      <td>0.913043</td>
      <td>0.351767</td>
    </tr>
    <tr>
      <th>3</th>
      <td>13</td>
      <td>5.661497</td>
      <td>33.993773</td>
      <td>0.960526</td>
      <td>0.406547</td>
    </tr>
    <tr>
      <th>4</th>
      <td>14</td>
      <td>2.815029</td>
      <td>13.459270</td>
      <td>1.000000</td>
      <td>0.461663</td>
    </tr>
    <tr>
      <th>5</th>
      <td>15</td>
      <td>4.812033</td>
      <td>31.726726</td>
      <td>1.000000</td>
      <td>0.508752</td>
    </tr>
  </tbody>
</table>
</div>


RMSE      6.912953\
Score    62.022741\
PICP      0.978928\
PINAW     0.470662

# DS03 (normal-filtered-degradation)


<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>unit</th>
      <th>RMSE</th>
      <th>Score</th>
      <th>PICP</th>
      <th>PINAW</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>10</td>
      <td>2.287758</td>
      <td>8.309579</td>
      <td>1.000000</td>
      <td>0.549712</td>
    </tr>
    <tr>
      <th>1</th>
      <td>11</td>
      <td>5.867247</td>
      <td>24.833282</td>
      <td>1.000000</td>
      <td>0.640541</td>
    </tr>
    <tr>
      <th>2</th>
      <td>12</td>
      <td>7.713505</td>
      <td>38.687416</td>
      <td>0.982143</td>
      <td>0.343562</td>
    </tr>
    <tr>
      <th>3</th>
      <td>13</td>
      <td>6.360628</td>
      <td>30.876775</td>
      <td>0.948276</td>
      <td>0.417969</td>
    </tr>
    <tr>
      <th>4</th>
      <td>14</td>
      <td>1.265477</td>
      <td>4.052291</td>
      <td>1.000000</td>
      <td>0.480910</td>
    </tr>
    <tr>
      <th>5</th>
      <td>15</td>
      <td>2.201658</td>
      <td>5.400241</td>
      <td>1.000000</td>
      <td>0.516345</td>
    </tr>
  </tbody>
</table>
</div>


RMSE      4.282712\
Score    18.693264\
PICP      0.988403\
PINAW     0.491506

# DS03 (normal-filtered-benchmark)
<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>unit</th>
      <th>RMSE</th>
      <th>Score</th>
      <th>PICP</th>
      <th>PINAW</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>10</td>
      <td>4.190528</td>
      <td>26.301490</td>
      <td>1.000000</td>
      <td>0.509070</td>
    </tr>
    <tr>
      <th>1</th>
      <td>11</td>
      <td>10.147794</td>
      <td>95.884897</td>
      <td>1.000000</td>
      <td>0.586171</td>
    </tr>
    <tr>
      <th>2</th>
      <td>12</td>
      <td>11.277824</td>
      <td>85.389164</td>
      <td>0.878788</td>
      <td>0.357122</td>
    </tr>
    <tr>
      <th>3</th>
      <td>13</td>
      <td>6.011947</td>
      <td>32.141085</td>
      <td>0.954545</td>
      <td>0.419257</td>
    </tr>
    <tr>
      <th>4</th>
      <td>14</td>
      <td>2.972703</td>
      <td>12.721016</td>
      <td>1.000000</td>
      <td>0.487072</td>
    </tr>
    <tr>
      <th>5</th>
      <td>15</td>
      <td>4.812033</td>
      <td>31.726726</td>
      <td>1.000000</td>
      <td>0.508752</td>
    </tr>
  </tbody>
</table>
</div>

RMSE      6.568805\
Score    47.360730\
PICP      0.972222\
PINAW     0.477907

## Old Results

## DS01

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>unit</th>
      <th>RMSE</th>
      <th>Score</th>
      <th>PICP</th>
      <th>PINAW</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>7</td>
      <td>5.775061</td>
      <td>55.267201</td>
      <td>0.999001</td>
      <td>0.309502</td>
    </tr>
    <tr>
      <th>1</th>
      <td>8</td>
      <td>3.785441</td>
      <td>57.762069</td>
      <td>1.000000</td>
      <td>0.295153</td>
    </tr>
    <tr>
      <th>2</th>
      <td>9</td>
      <td>6.018379</td>
      <td>99.245554</td>
      <td>1.000000</td>
      <td>0.428916</td>
    </tr>
    <tr>
      <th>3</th>
      <td>10</td>
      <td>5.708555</td>
      <td>92.050901</td>
      <td>0.998796</td>
      <td>0.415006</td>
    </tr>
  </tbody>
</table>
</div>

RMSE:	5.321859\
Score:	76.081431\
PICP:	0.999449\
PINAW:	0.362144

## DS03

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>unit</th>
      <th>RMSE</th>
      <th>Score</th>
      <th>PICP</th>
      <th>PINAW</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>10</td>
      <td>1.970763</td>
      <td>16.453508</td>
      <td>1.000000</td>
      <td>0.383870</td>
    </tr>
    <tr>
      <th>1</th>
      <td>11</td>
      <td>7.169923</td>
      <td>104.176352</td>
      <td>0.991818</td>
      <td>0.430604</td>
    </tr>
    <tr>
      <th>2</th>
      <td>12</td>
      <td>15.045545</td>
      <td>382.033190</td>
      <td>0.507948</td>
      <td>0.242492</td>
    </tr>
    <tr>
      <th>3</th>
      <td>13</td>
      <td>6.132815</td>
      <td>44.879145</td>
      <td>0.987355</td>
      <td>0.315902</td>
    </tr>
    <tr>
      <th>4</th>
      <td>14</td>
      <td>3.357752</td>
      <td>23.227942</td>
      <td>0.998947</td>
      <td>0.370075</td>
    </tr>
    <tr>
      <th>5</th>
      <td>15</td>
      <td>2.875275</td>
      <td>18.082909</td>
      <td>1.000000</td>
      <td>0.376594</td>
    </tr>
  </tbody>
</table>
</div>


RMSE:	6.092012\
Score:	98.142175\
PICP:	0.914345\
PINAW:	0.353256

## DS05 

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>unit</th>
      <th>RMSE</th>
      <th>Score</th>
      <th>PICP</th>
      <th>PINAW</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>7</td>
      <td>9.023385</td>
      <td>129.525538</td>
      <td>0.996078</td>
      <td>0.370195</td>
    </tr>
    <tr>
      <th>1</th>
      <td>8</td>
      <td>11.351902</td>
      <td>207.492181</td>
      <td>0.998291</td>
      <td>0.339061</td>
    </tr>
    <tr>
      <th>2</th>
      <td>9</td>
      <td>3.117663</td>
      <td>23.322696</td>
      <td>0.997016</td>
      <td>0.408783</td>
    </tr>
    <tr>
      <th>3</th>
      <td>10</td>
      <td>7.332727</td>
      <td>102.286471</td>
      <td>1.000000</td>
      <td>0.334772</td>
    </tr>
  </tbody>
</table>
</div>

RMSE:       7.706419\
Score:    115.656722\
PICP:       0.997846\
PINAW:      0.363203